# 추천시스템 기초 실습

## 0. 데이터 불러오기 및 탐색

데이터는 movielens에서 제공하는 데이터 중 하나로, 가장 크기가 작은 버전으로 가져왔습니다.
- movies.csv: 영화 제목과 영화 장르에 대한 데이터
- ratings.csv: user(사용자)-movie(아이템) 선호도 데이터
- tags.csv: user가 해당 movie에 부여한 tag 데이터

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DIR = '/content/drive/MyDrive/Colab Notebooks/tobigs15/'

## 1. Content-based Filtering 실습

content-based filtering은 movie(아이템) 간의 유사도를 계산하여 해당 movie와 유사한 movie를 추천해주는 간단한 추천 알고리즘

제공된 데이터에서 movie의 콘텐츠를 확인할 수 있는 데이터는 movies.csv의 장르와 tags.csv의 태그 정보가 있음

In [4]:
# 아이템 간 유사도 계산을 위한 데이터프레임 만들기
# 열은 영화 장르가 될 수도 있고, 영화 태그가 될 수도 있음 (영화 태그의 경우 모든 영화에 대한 태그는 없음)
movies = pd.read_csv(DIR+"ml-latest-small/movies.csv")
tags = pd.read_csv(DIR+"ml-latest-small/tags.csv")
ratings = pd.read_csv(DIR+"ml-latest-small/ratings.csv")
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


![image.png](attachment:image.png)

In [5]:
piv = pd.DataFrame(columns = ["movieId"])

for i in range(len(movies)):
    # print(m.movieId)
    genres = movies.loc[i, 'genres'].split("|") # genre를 |로 분리하고, 이에 해당하는 칸을 1로 설정하였다.
    piv.loc[i, 'movieId'] = movies.loc[i, 'movieId']
    piv.loc[i, genres] = 1
piv = piv.set_index("movieId")
piv

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# TF-IDF를 적용할 수 있지만 여기서는 skip
# TF-IDF를 적용하게 되면 다음과 같이 나오긴 합니당

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
movies['corpus'] = movies['genres'].apply(lambda x : ' '.join(x.split('|')))

In [9]:
tfidv = TfidfVectorizer().fit(movies['corpus'])

In [10]:
temp = pd.DataFrame(tfidv.transform(movies['corpus']).toarray())

In [11]:
temp

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [12]:
# 아이템 간 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
# 영화-영화의 유사도를 코사인 유사도를 사용해서 구해줍니당
CBF = pd.DataFrame(cosine_similarity(temp)) # cosine similarity 함수를 사용한 후 보기 편하도록 DataFrame으로 변환하였습니다.

In [14]:
# movie - moive 아이템 간 유사도 데이터프레임
CBF.index = piv.index
CBF.columns = piv.index
CBF

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.813578,0.152769,0.135135,0.267586,0.000000,0.152769,0.654698,0.000000,0.262413,0.135135,0.136349,0.833737,0.000000,0.257331,0.000000,0.000000,0.267586,0.267586,0.094253,0.115861,0.000000,0.000000,0.000000,0.000000,0.000000,0.453434,0.000000,0.379305,0.000000,0.000000,0.0,0.453434,0.000000,0.571377,0.152769,0.000000,0.000000,0.000000,0.000000,...,0.0,0.157867,0.267586,0.264947,0.205390,0.182414,0.763343,0.091411,0.435076,0.0,0.361137,0.473788,0.000000,0.152769,0.267586,0.691840,0.196578,0.0,0.262413,0.000000,0.000000,0.128103,0.0,0.000000,0.267586,0.000000,0.000000,0.516225,0.0,0.276524,0.360397,0.465621,0.196578,0.516225,0.0,0.680258,0.755891,0.000000,0.421037,0.267586
2,0.813578,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.804715,0.000000,0.322542,0.000000,0.000000,0.631907,0.000000,0.316295,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.557333,0.000000,0.466219,0.000000,0.000000,0.0,0.557333,0.000000,0.548271,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.325657,0.252453,0.224212,0.578554,0.000000,0.534769,0.0,0.339639,0.495234,0.000000,0.000000,0.000000,0.723156,0.000000,0.0,0.322542,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.240700,0.000000,0.000000,0.000000,0.000000,0.0,0.341376,0.379331,0.000000,0.000000,0.000000
3,0.152769,0.000000,1.000000,0.884571,0.570915,0.000000,1.000000,0.000000,0.000000,0.000000,0.884571,0.290910,0.000000,0.000000,0.467874,0.000000,0.690816,0.570915,0.570915,0.201096,0.247197,0.000000,0.000000,0.000000,0.690816,0.000000,0.000000,0.690816,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.267370,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.336821,0.570915,0.000000,0.000000,0.000000,0.000000,0.195033,0.000000,0.0,0.180957,0.151220,0.000000,1.000000,0.570915,0.220816,0.419413,0.0,0.000000,0.000000,0.000000,0.273316,0.0,0.690816,0.570915,0.000000,0.000000,0.000000,0.0,0.172171,0.162848,0.000000,0.419413,0.000000,0.0,0.181883,0.202105,0.000000,0.000000,0.570915
4,0.135135,0.000000,0.884571,1.000000,0.505015,0.000000,0.884571,0.000000,0.000000,0.000000,1.000000,0.257331,0.000000,0.466405,0.413868,0.234877,0.863110,0.505015,0.505015,0.329607,0.218663,0.124930,0.000000,0.168322,0.863110,0.466405,0.205061,0.863110,0.107408,0.234877,0.466405,0.0,0.205061,0.234877,0.236508,0.884571,0.466405,0.182705,0.188238,0.466405,...,0.0,0.297942,0.505015,0.000000,0.000000,0.121002,0.000000,0.172521,0.000000,0.0,0.160070,0.133765,0.234877,0.884571,0.505015,0.195327,0.687440,0.0,0.000000,0.000000,0.000000,0.447980,0.0,0.863110,0.505015,0.466405,0.466405,0.000000,0.0,0.152298,0.144051,0.201391,0.687440,0.000000,0.0,0.160888,0.178776,0.466405,0.000000,0.505015
5,0.267586,0.000000,0.570915,0.505015,1.000000,0.000000,0.570915,0.000000,0.000000,0.000000,0.505015,0.509550,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.352234,0.432984,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.468319,0.570915,0.000000,0.000000,0.000000,0.000000,...,0.0,0.589966,1.000000,0.000000,0.000000,0.000000,0.000000,0.341614,0.000000,0.0,0.316960,0.264872,0.000000,0.570915,1.000000,0.386775,0.734632,0.0,0.000000,0.000000,0.000000,0.478734,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.301570,0.285240,0.000000,0.734632,0.000000,0.0,0.318581,0.354002,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [15]:
# 각 영화와 다른 영화와의 유사도가 모두 계산되어 있고, 유사도가 높은 영화가 가장 비슷한 영화라고 할 수 있습니당
# 입력 아이템과 가장 유사한 아이템 TOP10개를 뽑아봅시당
def getCBF(movieId):
    print("입력 영화: {}".format(movies[movies.movieId==movieId].title.item()))
    print("입력 영화 장르: {}".format(movies[movies.movieId==movieId].genres.item()))
    movie_list = CBF.loc[movieId].nlargest(10) # 해당 movieId 행에서 유사도가 높은 순서대로 10개를 뽑는다.
    top10 = pd.DataFrame(columns = movies.columns)
    for i in movie_list.index:
        # print(movies[movies["movieId"]==i])
        top10 = pd.concat([top10, movies[movies["movieId"]==i]])    # top10 DataFrame에 유사도가 높은 movie를 하나씩 추가한다.
    print("=====Most similar List=====")
    return top10.set_index('movieId')

In [16]:
getCBF(193583)

입력 영화: No Game No Life: Zero (2017)
입력 영화 장르: Animation|Comedy|Fantasy
=====Most similar List=====


,title,genres,corpus
movieId,,,
4158,Monkeybone (2001),Animation|Comedy|Fantasy,Animation Comedy Fantasy
6773,"Triplets of Belleville, The (Les triplettes de...",Animation|Comedy|Fantasy,Animation Comedy Fantasy
7228,Cool World (1992),Animation|Comedy|Fantasy,Animation Comedy Fantasy
72692,Mickey's Once Upon a Christmas (1999),Animation|Comedy|Fantasy,Animation Comedy Fantasy
79008,South Park: Imaginationland (2008),Animation|Comedy|Fantasy,Animation Comedy Fantasy
126577,"Daddy, I'm A Zombie (2012)",Animation|Comedy|Fantasy,Animation Comedy Fantasy
139855,Anomalisa (2015),Animation|Comedy|Fantasy,Animation Comedy Fantasy
182297,Porky in Wackyland (1938),Animation|Comedy|Fantasy,Animation Comedy Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,Animation Comedy Fantasy


영화간의 유사도를 비교하여 추천 영화를 추출한 결과, 쿼리로 주어진 영화의 장르인 Animation, Comedy, Fantasy장르의 영화들이 결과로 나온 것을 확인할 수 있다.

## 2. Collaborative Filtering 실습

User-based CF를 실습해 봅시당

In [17]:
# User-Item 간 선호도 데이터프레임 만들기
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


![image.png](attachment:image.png)

In [28]:
# 해당 데이터프레임은 사용자-영화 선호도 데이터프레임입니당
piv = pd.pivot_table(ratings, index = 'userId', columns = 'movieId', values = 'rating')
for mov in np.unique(movies.movieId):
    if mov not in piv.columns:  # 어떤 사용자도 보지 않은 영화가 있다면
        piv[mov] = np.nan     # NaN 열을 추가한다.
piv

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609,1076,2939,3338,3456,4194,5721,6668,6849,7020,7792,8765,25855,26085,30892,32160,32371,34482,85565
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,2.5,NaN,NaN,NaN,3.5,NaN,4.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,4.5,NaN,NaN,4.0,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,2.0,NaN,3.5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.5,3.5,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# 사용자 간 유사도 구하기
CF = pd.DataFrame(cosine_similarity(piv.fillna(0)))

In [30]:
# 사용자와 사용자의 유사도를 구할 때 꼭 코사인 유사도를 사용하지 않아도 됩니당
# 보완 코사인 유사도 혹은 피어슨 상관계수를 적용해보아유
CF.index = piv.index
CF.columns = piv.index
CF

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.027283,0.059720,0.194395,0.129080,0.128152,0.158744,0.136968,0.064263,0.016875,0.132499,0.016458,0.092971,0.113238,0.160689,0.169858,0.264358,0.214868,0.325376,0.160969,0.153162,0.050691,0.106669,0.155193,0.099261,0.105791,0.238933,0.202866,0.145251,0.099781,0.164454,0.146211,0.151945,0.083544,0.074508,0.071673,0.144736,0.138588,0.329782,0.094755,...,0.090224,0.131072,0.253480,0.093450,0.062668,0.051803,0.317319,0.000000,0.141129,0.249326,0.057399,0.048914,0.052175,0.100406,0.114076,0.123280,0.122782,0.183922,0.118112,0.324766,0.136809,0.143934,0.174413,0.141960,0.110558,0.123713,0.312843,0.011280,0.282412,0.291272,0.080554,0.164455,0.221486,0.070669,0.153625,0.164191,0.269389,0.291097,0.093572,0.145321
2,0.027283,1.000000,0.000000,0.003726,0.016614,0.025333,0.027585,0.027257,0.000000,0.067445,0.044419,0.000000,0.043918,0.016901,0.119778,0.093728,0.103755,0.166253,0.012571,0.014137,0.090880,0.144635,0.013597,0.129902,0.226008,0.000000,0.000000,0.058145,0.104059,0.149324,0.000000,0.017807,0.032428,0.043299,0.000000,0.054012,0.028500,0.023147,0.000000,0.017446,...,0.000000,0.032675,0.167959,0.035531,0.000000,0.000000,0.014870,0.000000,0.000000,0.061953,0.136703,0.219315,0.043620,0.000000,0.127551,0.089562,0.000000,0.055900,0.026429,0.056348,0.036147,0.030684,0.062033,0.013125,0.000000,0.104568,0.011986,0.048508,0.098000,0.023248,0.202671,0.016866,0.011997,0.000000,0.000000,0.028429,0.012948,0.046211,0.027565,0.102427
3,0.059720,0.000000,1.000000,0.002251,0.005020,0.003936,0.000000,0.004941,0.000000,0.000000,0.000000,0.000000,0.000000,0.003064,0.017251,0.032299,0.009813,0.028241,0.019142,0.008756,0.004017,0.003070,0.003081,0.002592,0.005100,0.000000,0.002332,0.029771,0.005974,0.000000,0.003507,0.015494,0.007156,0.000736,0.000000,0.000000,0.000000,0.004196,0.003639,0.002530,...,0.020899,0.008056,0.000000,0.000000,0.000000,0.029830,0.039894,0.000000,0.005749,0.033121,0.008561,0.000000,0.000000,0.000000,0.000000,0.000986,0.015793,0.003923,0.004791,0.027418,0.000000,0.000000,0.006460,0.001983,0.000000,0.025873,0.028970,0.000000,0.039539,0.013143,0.005048,0.004892,0.024992,0.000000,0.010694,0.012993,0.019247,0.021128,0.000000,0.032119
4,0.194395,0.003726,0.002251,1.000000,0.128659,0.088491,0.115120,0.062969,0.011361,0.031163,0.054767,0.049945,0.076949,0.048989,0.071551,0.164761,0.145058,0.123217,0.206053,0.113755,0.053014,0.043403,0.093971,0.071690,0.032862,0.061832,0.128954,0.135593,0.061308,0.058140,0.126182,0.200597,0.167247,0.045571,0.050148,0.113807,0.061031,0.090125,0.240976,0.053436,...,0.018553,0.182857,0.125627,0.064559,0.061035,0.005932,0.169440,0.000000,0.098777,0.148584,0.043871,0.047619,0.048474,0.041246,0.088289,0.064095,0.138766,0.049146,0.064485,0.219298,0.116111,0.068196,0.217116,0.082614,0.091974,0.107908,0.275436,0.016054,0.201610,0.211921,0.085938,0.128273,0.307973,0.052985,0.084584,0.200395,0.131746,0.149858,0.032198,0.107683
5,0.129080,0.016614,0.005020,0.128659,1.000000,0.300349,0.108342,0.429075,0.000000,0.030611,0.183805,0.058860,0.017157,0.221711,0.110152,0.082171,0.162633,0.121313,0.098758,0.096474,0.058264,0.033074,0.066889,0.096215,0.040705,0.294282,0.100491,0.121894,0.068876,0.065534,0.068585,0.233932,0.282323,0.029953,0.311472,0.022065,0.303078,0.377773,0.152956,0.321077,...,0.020016,0.124806,0.129338,0.341347,0.049306,0.000000,0.122199,0.000000,0.182382,0.117199,0.057538,0.000000,0.124645,0.348215,0.029293,0.072713,0.154315,0.377256,0.230961,0.152971,0.000000,0.359595,0.171864,0.137990,0.073238,0.096181,0.116071,0.000000,0.098599,0.137053,0.068048,0.418747,0.110148,0.258773,0.148758,0.106435,0.152866,0.135535,0.261232,0.060792


In [31]:
# 입력 사용자와 가장 가까운 유저에서 선호도가 가장 높은 아이템 TOP5를 추천해봅시당!
# 이 때, 입력 사용자가 이미 시청한/평가한 영화는 제외해야하도록 추천 리스트를 만들어봅시당
import math

def getCF(userId):
    most_similar_user = np.argmax(CF.loc[userId])   # 입력사용자와 가장 유사한 유저를 뽑는다.
    top_ratings = np.argsort(piv.loc[most_similar_user].fillna(0))[::-1]    # 유사 사용자가 rating을 매긴 것 중 높은 순으로 movieId를 정렬한다.
    top5 = pd.DataFrame(columns = movies.columns)
    for i in top_ratings:
        if len(top5) ==5:
            break
        if math.isnan(piv.loc[userId][i]):  # 사용자가 이전에 rating을 매기지 않은 경우만
            top5 = pd.concat([top5, movies[movies["movieId"]==i]])    # top5 DataFrame에 rating이 높은 movie를 하나씩 추가한다.
    print("=====Recommendation List=====")
    return top5.set_index('movieId')

In [32]:
getCF(604)

=====Recommendation List=====


,title,genres,corpus
movieId,,,
515,"Remains of the Day, The (1993)",Drama|Romance,Drama Romance
840,House Arrest (1996),Children|Comedy,Children Comedy
2144,Sixteen Candles (1984),Comedy|Romance,Comedy Romance
928,Rebecca (1940),Drama|Mystery|Romance|Thriller,Drama Mystery Romance Thriller
932,"Affair to Remember, An (1957)",Drama|Romance,Drama Romance


604번 사용자와 유사한 유저의 상위 rating 값 중 604번 사용자가 아직 보지 않은 영화를 추천한 결과이다. 앞선 결과와는 다르게, 다양한 장르의 영화가 추천된 것을 확인할 수 있다. 

## 3. Matrix Factorization 실습

같은 폴더에 있는 MF.py의 MatrixFactorization()을 적용해 봅시당

In [33]:
import sys
sys.path.append(DIR)

import MF

In [34]:
# 사용자-영화 선호도 데이터프레임입니당
piv.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609,1076,2939,3338,3456,4194,5721,6668,6849,7020,7792,8765,25855,26085,30892,32160,32371,34482,85565
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# MF.py를 참고하여 인자를 입력해 보아유
factorizer = MF.MatrixFactorization(np.array(piv.fillna(0)), k=100, learning_rate=0.01, reg_param=0.01, epochs=200, verbose=True)
"""
R: user-Movie간의 rating 행렬
k: latent의 개수: 임의로 100으로 설정
"""
factorizer.fit()
R_reduced = factorizer.print_results()

Iteration: 10 ; cost = 0.0024
Iteration: 20 ; cost = 0.0021
Iteration: 30 ; cost = 0.0020
Iteration: 40 ; cost = 0.0018
Iteration: 50 ; cost = 0.0017
Iteration: 60 ; cost = 0.0016
Iteration: 70 ; cost = 0.0015
Iteration: 80 ; cost = 0.0014
Iteration: 90 ; cost = 0.0013
Iteration: 100 ; cost = 0.0013
Iteration: 110 ; cost = 0.0012
Iteration: 120 ; cost = 0.0011
Iteration: 130 ; cost = 0.0011
Iteration: 140 ; cost = 0.0011
Iteration: 150 ; cost = 0.0010
Iteration: 160 ; cost = 0.0010
Iteration: 170 ; cost = 0.0009
Iteration: 180 ; cost = 0.0009
Iteration: 190 ; cost = 0.0009
Iteration: 200 ; cost = 0.0009
User Latent P:
[[-0.12891329 -0.16660148  0.17200642 ... -0.31042386 -0.01662172
   0.09057833]
 [ 0.02861807  0.13351684 -1.04943078 ...  0.12289567 -0.17743283
   0.14507268]
 [ 0.23733206 -0.03484917  0.15468689 ... -0.2660257   0.36893373
  -0.22306497]
 ...
 [ 0.17244818  0.18736696 -0.28477706 ... -0.21038304  0.01091063
  -0.16414574]
 [-0.18276056 -0.38956692 -0.05870956 ...  0.

In [36]:
# Matrix Factorizaion으로 찾아낸 잠재요인을 기반으로 재생산한 행렬이네유
R_reduced = pd.DataFrame(R_reduced)
R_reduced

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9702,9703,9704,9705,9706,9707,9708,9709,9710,9711,9712,9713,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723,9724,9725,9726,9727,9728,9729,9730,9731,9732,9733,9734,9735,9736,9737,9738,9739,9740,9741
0,4.215796,3.224931,3.967980,4.175243,4.207083,3.889886,3.520897,5.332156,3.318095,3.828755,4.019672,2.301153,2.161214,4.660634,2.222532,4.420931,4.053109,1.613495,3.565180,4.009523,4.116057,4.150353,4.335226,4.722665,3.536366,2.305268,2.138831,3.371934,4.918367,6.837382,2.568568,5.300958,4.487545,4.559020,5.551047,4.295594,7.153220,3.921221,2.806274,2.550315,...,5.362567,5.350170,7.426979,2.784827,5.493251,2.121759,3.621504,8.548076,2.166154,2.440984,0.235826,4.412678,6.109358,2.021358,12.662430,5.139008,5.867409,3.353468,4.235571,5.220429,2.048535,1.758596,5.320268,3.621423,3.333775,4.580731,2.492370,1.224326,3.117415,5.845304,4.081709,7.950498,6.011818,6.439407,5.669621,7.566310,0.699369,6.734534,4.615412,-1.202474
1,3.933506,1.633423,3.580024,-0.209946,1.875307,2.769146,3.599119,7.514915,4.567758,4.148458,4.913494,1.580104,4.773358,5.528453,2.872181,2.776711,3.842303,4.624334,1.918359,4.368957,4.124190,2.972816,10.672645,0.347415,3.465599,0.555273,2.933355,-0.508017,1.323421,7.760276,3.217128,2.317727,2.460557,3.550561,2.788742,3.487609,1.051851,5.024926,2.166558,-1.200597,...,2.743818,8.991046,10.564579,5.609909,10.092589,5.649239,-4.532881,5.577908,3.535809,6.448157,3.620000,0.355600,1.382116,4.919337,6.080505,4.952143,8.546790,3.520068,2.954511,-1.899338,6.651771,10.165138,-5.817739,6.378915,6.632653,9.816340,6.234304,7.220383,7.879941,3.856402,4.058985,0.241255,13.007848,-1.541566,-1.868889,3.327208,6.987731,-2.223710,1.403589,2.634530
2,1.782991,3.206417,2.686962,5.607762,3.774146,2.864446,0.483633,-1.049318,3.664080,1.964150,1.624599,1.779673,1.612965,4.797946,4.860517,2.079212,2.837375,4.409586,2.699827,-1.074404,0.821140,1.630824,0.885151,3.971530,1.304131,6.101328,3.125426,2.107812,3.504363,2.616760,0.524494,5.442582,3.312533,3.864632,1.953104,2.238204,7.255297,5.015731,5.336366,-4.116234,...,3.064892,-1.684147,2.035446,-0.575801,9.556753,2.970176,-5.136025,3.121221,6.587433,1.697167,4.338676,3.009950,10.770665,0.679137,9.669778,4.999878,-1.066207,3.974397,-1.508075,-0.034419,1.898685,-2.166550,2.650992,4.974295,6.524428,3.881990,0.922115,2.434028,-0.612341,2.846622,6.680456,3.891442,3.144201,-0.791070,0.173875,7.539140,-1.935324,3.346611,1.406661,5.525329
3,2.353303,3.448305,4.425407,3.809167,3.147225,3.205806,3.874848,1.315606,0.072250,3.687749,3.452184,3.287196,-4.981555,6.635064,2.811001,3.924925,4.018668,3.628780,1.979763,3.132716,3.043876,6.119314,1.705743,-0.239782,4.117462,6.654443,0.704328,2.545425,2.783765,1.715702,3.186652,2.115488,3.648391,4.408465,1.355746,3.219409,2.986210,2.494614,-1.894799,4.316992,...,-3.662198,2.595452,9.939620,4.199160,-1.985753,4.140680,2.033172,2.998283,7.553637,2.571796,5.949241,4.817831,4.645110,3.196026,1.021791,2.865827,-0.582216,0.213841,3.189696,4.671468,0.828780,2.020631,-1.648916,5.764719,3.561376,-3.926314,5.345725,5.014603,2.519290,5.544603,4.980535,-1.547016,11.511192,6.583938,-0.165280,1.965773,4.343255,2.386791,2.217595,5.992108
4,4.026791,2.663772,4.328703,1.643235,3.512028,3.008795,3.965978,3.115013,4.294747,3.136258,2.750857,5.646312,3.047890,3.948188,5.884659,5.051237,3.272028,4.705826,2.249095,7.368861,3.874947,4.492914,2.965881,1.583175,4.051550,8.787897,1.347210,4.368228,3.573315,3.123475,2.201531,2.976060,3.920065,3.989119,5.025768,3.055421,6.278759,4.562359,1.710192,-0.840923,...,6.032731,1.534910,2.146349,4.092998,-2.548922,6.567513,0.337235,4.382758,0.448541,0.383365,-0.367445,8.340846,5.190435,3.135311,5.237238,-1.881708,7.752216,6.963967,5.934189,0.491408,1.745939,1.838984,-0.170547,9.357088,5.847352,7.176047,6.659457,10.066191,-0.664182,3.806433,2.604713,3.319296,7.356332,5.749020,1.684511,5.150289,2.864860,3.459954,-3.533274,5.353801
...,..

In [37]:
def recommend_by_MF(userId, rank=10):
    top_Rating = np.argsort(R_reduced[userId])[::-1]  # 재생산한 Rate Matrix에서 해당 user의 movie에 대한 rating값을 높은 순으로 정렬한다.
    top = pd.DataFrame(columns = movies.columns)
    for i in top_Rating:
        if len(top) ==rank:
            break
        if math.isnan(piv.loc[userId][i]):  # 사용자가 이전에 rating을 매기지 않은 경우만
            top = pd.concat([top, movies[movies["movieId"]==i]])    # top DataFrame에 rating이 높은 movie를 하나씩 추가한다.
    print("=====Recommendation List=====")
    return top.set_index('movieId')

recommend_by_MF(604, 5)

=====Recommendation List=====


,title,genres,corpus
movieId,,,
548,Terminal Velocity (1994),Action|Mystery|Thriller,Action Mystery Thriller
441,Dazed and Confused (1993),Comedy,Comedy
430,Calendar Girl (1993),Comedy|Drama,Comedy Drama
206,Unzipped (1995),Documentary,Documentary
377,Speed (1994),Action|Romance|Thriller,Action Romance Thriller


MF를 통해 추정한 Rate Matrix를 통해 사용자의 id를 입력하면 영화를 추천해주는 함수를 구현하였다. 앞선 추천결과와는 상이한 결과를 보였지만 대부분의 장르가 Thiller, Comedy, Drama 등 604번 사용자가 선호하는 장르임을 확인할 수 있다.

## 4. 본인만의 추천 리스트 만들어보기

배운 내용 혹은 알고 있는 추천 알고리즘을 사용해서 user의 다음 추천 아이템에 대한 TOP10 추천 리스트를 만들어 봅시당

코드를 이해할 수 있도록 주석을 꼼꼼하게 달아주시면 감사하겠습니다!

In [38]:
!pip install scikit-surprise
import surprise

     |████████████████████████████████| 11.8MB 290kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617589 sha256=44e1b30334476ea67a4ee0b15cdb4440bd8f5c5b5219aeaf7595a1661c126a76
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


추천시스템에 많이 사용되는 surprise 패키지를 import한다.

In [39]:
reader = surprise.Reader(rating_scale=(1,5))    # 데이터를 읽어들이기 위한 Reader 객체를 선언한다.
col_list = ['userId', 'movieId', 'rating']
data = surprise.Dataset.load_from_df(ratings[col_list], reader)   # dataframe으로부터 surprise용 dataset으로 변환한다.
trainset = data.build_full_trainset()   # data전체를 training에 사용한다.
algo = surprise.KNNBasic(sim_options = {'name': 'pearson'}).fit(trainset)  # Nearset Neigbor 방식으로 추천을 한다. 이 때, pearson을 계산하여 Nearest를 판단한다.

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [40]:
def recommend_by_KNN(userId, k=5, rank=10):
    unseen = pd.DataFrame(columns = ['movieId', 'rating'])
    for movie in np.unique(piv.columns):
        if math.isnan(piv.loc[userId][movie]):  # 사용자가 이전에 rating을 매기지 않은 경우만
            rating = algo.predict(userId, movie)[3] # rating 값을 예측한다.
            unseen = unseen.append({'movieId': movie, 'rating': rating}, ignore_index=True) # unseen DataFrame에 예측한 rating값을 추가한다.
    unseen = unseen.sort_values(by=['rating'], axis=0, ascending=False) # 예측된 rating이 높은 순서대로 정렬한다.

    top = pd.DataFrame(columns = movies.columns)
    for i in unseen.movieId:
        if len(top) ==rank:
            break
        top = pd.concat([top, movies[movies["movieId"]==i]])    # top DataFrame에 rating이 높은 movie를 하나씩 추가한다.
    print("=====Recommendation List=====")
    return top.set_index('movieId')

recommend_by_KNN(604, rank = 5)

=====Recommendation List=====


,title,genres,corpus
movieId,,,
173351,Wow! A Talking Fish! (1983),Animation|Children|Comedy|Fantasy,Animation Children Comedy Fantasy
5490,The Big Bus (1976),Action|Comedy,Action Comedy
6201,Lady Jane (1986),Drama|Romance,Drama Romance
5513,Martin Lawrence Live: Runteldat (2002),Comedy|Documentary,Comedy Documentary
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,Romance


KNN을 이용하여 604번 사용자에게 추천하는 목록을 뽑는 함수를 구현하였다. 604번 사용자와 유사한 사용자들의 데이터를 각 영화에 대해 604번 사용자가 부여할 평점을 예측한다. 그리고, 604번 사용자가 기존에 보지 않은 영화 중에서 평점이 높은 순서대로 추천은 진행한다. 그 결과, 앞선 추천보다 훨씬 다양한 장르의 영화에 대한 추천을 얻을 수 있었다. 다만, "Wow! A Talking Fish!"와 같이 직관적으로 이해되지 않는 추천결과도 있었다. 아마 대중적으로 유명한 영화여서 추천결과에 포함된 것으로 추측된다.